In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time
import random
import os
from dotenv import load_dotenv, dotenv_values

In [2]:
team = 'ARI'
year = '2023'
URL = f'https://www.footballguys.com/stats/game-logs-against/teams?team={team}&year={year}'

In [3]:
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'lxml')

In [4]:
teams_raw = soup.find('select',{'class':'big-select'})
teams_options = teams_raw.find_all('option')
teams = []
for team in teams_options:
    teams.append(team['value'])

In [5]:
tables = soup.find_all('table',{'class':'table sortable-table'})
df = pd.read_html(StringIO(str(tables[0])))[0]
df = df[0:0]

In [6]:
def table_parser(df, team, year):
    URL = f'https://www.footballguys.com/stats/game-logs-against/teams?team={team}&year={year}'
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'lxml')
    tables = soup.find_all('table',{'class':'table sortable-table'})
    temp_df = pd.read_html(StringIO(str(tables[0])))[0]
    temp_df['pos'] = 'qb'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    temp_df = pd.read_html(StringIO(str(tables[1])))[0]
    temp_df['pos'] = 'rb'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    temp_df = pd.read_html(StringIO(str(tables[2])))[0]
    temp_df['pos'] = 'wr'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    temp_df = pd.read_html(StringIO(str(tables[3])))[0]
    temp_df['pos'] = 'te'
    temp_df['opp'] = team
    df = pd.concat([df, temp_df])
    return df

In [7]:
for team in teams:
    df = table_parser(df, team, year)
    time.sleep(random.uniform(2,5))

In [8]:
df['team-pos'] = df['team'] + '-' + df['pos']

In [9]:
df_sum_teampos = df.copy().drop(columns=['name'])
df_sum_teampos = df_sum_teampos.groupby(by=['team', 'pos', 'week', 'opp', 'team-pos']).sum()
df_sum_teampos = df_sum_teampos.reset_index()
df_sum_teampos.head(5)

,team,pos,week,opp,team-pos,cmp,att,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd
0,ARI,qb,1,WAS,ARI-qb,21.0,30.0,132.0,0.0,0.0,3.0,-3.0,0.0,0.0,0.0,0.0,0.0
1,ARI,qb,2,NYG,ARI-qb,21.0,31.0,228.0,1.0,0.0,3.0,41.0,1.0,0.0,0.0,0.0,0.0
2,ARI,qb,3,DAL,ARI-qb,17.0,21.0,189.0,1.0,0.0,6.0,55.0,0.0,0.0,0.0,0.0,0.0
3,ARI,qb,4,SF,ARI-qb,28.0,41.0,265.0,2.0,0.0,12.0,48.0,0.0,0.0,0.0,0.0,0.0
4,ARI,qb,5,CIN,ARI-qb,15.0,32.0,166.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_avg_teampos = df_sum_teampos.copy().drop(columns=['opp', 'week'])
df_avg_teampos = df_avg_teampos.groupby(by=['team', 'pos', 'team-pos']).mean()
df_avg_teampos = df_avg_teampos.reset_index()
df_avg_teampos.head(5)

,team,pos,team-pos,cmp,att,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd
0,ARI,qb,ARI-qb,19.818182,31.727273,190.363636,0.818182,0.818182,6.000000,33.727273,0.545455,0.000000,0.000000,0.000000,0.000000
1,ARI,rb,ARI-rb,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,76.000000,0.272727,3.545455,2.818182,13.545455,0.000000
2,ARI,te,ARI-te,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.090909,7.000000,64.363636,0.181818
3,ARI,wr,ARI-wr,0.000000,0.000000,0.000000,0.000000,0.000000,2.272727,16.272727,0.090909,17.090909,10.000000,112.454545,0.636364
4,ATL,qb,ATL-qb,20.200000,32.000000,223.800000,0.900000,0.700000,4.100000,21.800000,0.400000,0.000000,0.000000,0.000000,0.000000


In [11]:
df_strength_merge = pd.merge(df_sum_teampos, df_avg_teampos, on=['team-pos'], how='left')
df_strength_merge.head()

,team_x,pos_x,week,opp,team-pos,cmp_x,att_x,pyd_x,ptd_x,int_x,...,pyd_y,ptd_y,int_y,rsh_y,rshyd_y,rshtd_y,targ_y,rec_y,recyd_y,rectd_y
0,ARI,qb,1,WAS,ARI-qb,21.0,30.0,132.0,0.0,0.0,...,190.363636,0.818182,0.818182,6.0,33.727273,0.545455,0.0,0.0,0.0,0.0
1,ARI,qb,2,NYG,ARI-qb,21.0,31.0,228.0,1.0,0.0,...,190.363636,0.818182,0.818182,6.0,33.727273,0.545455,0.0,0.0,0.0,0.0
2,ARI,qb,3,DAL,ARI-qb,17.0,21.0,189.0,1.0,0.0,...,190.363636,0.818182,0.818182,6.0,33.727273,0.545455,0.0,0.0,0.0,0.0
3,ARI,qb,4,SF,ARI-qb,28.0,41.0,265.0,2.0,0.0,...,190.363636,0.818182,0.818182,6.0,33.727273,0.545455,0.0,0.0,0.0,0.0
4,ARI,qb,5,CIN,ARI-qb,15.0,32.0,166.0,2.0,2.0,...,190.363636,0.818182,0.818182,6.0,33.727273,0.545455,0.0,0.0,0.0,0.0


In [12]:
df_strength_raw = pd.DataFrame()
df_strength_raw['opp'] = df_strength_merge['opp']
df_strength_raw['pos'] = df_strength_merge['pos_x']
df_strength_raw['team'] = df_strength_merge['team_x']
df_strength_raw['week'] = df_strength_merge['week']
df_strength_raw['att'] = df_strength_merge['att_x'] - df_strength_merge['att_y']
df_strength_raw['cmp'] = df_strength_merge['cmp_x'] - df_strength_merge['cmp_y']
df_strength_raw['pyd'] = df_strength_merge['pyd_x'] - df_strength_merge['pyd_y']
df_strength_raw['ptd'] = df_strength_merge['ptd_x'] - df_strength_merge['ptd_y']
df_strength_raw['int'] = df_strength_merge['int_x'] - df_strength_merge['int_y']
df_strength_raw['rsh'] = df_strength_merge['rsh_x'] - df_strength_merge['rsh_y']
df_strength_raw['rshyd'] = df_strength_merge['rshyd_x'] - df_strength_merge['rshyd_y']
df_strength_raw['rshtd'] = df_strength_merge['rshtd_x'] - df_strength_merge['rshtd_y']
df_strength_raw['targ'] = df_strength_merge['targ_x'] - df_strength_merge['targ_y']
df_strength_raw['rec'] = df_strength_merge['rec_x'] - df_strength_merge['rec_y']
df_strength_raw['recyd'] = df_strength_merge['recyd_x'] - df_strength_merge['recyd_y']
df_strength_raw['rectd'] = df_strength_merge['rectd_x'] - df_strength_merge['rectd_y']
df_strength_raw.head()

,opp,pos,team,week,att,cmp,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd
0,WAS,qb,ARI,1,-1.727273,1.181818,-58.363636,-0.818182,-0.818182,-3.0,-36.727273,-0.545455,0.0,0.0,0.0,0.0
1,NYG,qb,ARI,2,-0.727273,1.181818,37.636364,0.181818,-0.818182,-3.0,7.272727,0.454545,0.0,0.0,0.0,0.0
2,DAL,qb,ARI,3,-10.727273,-2.818182,-1.363636,0.181818,-0.818182,0.0,21.272727,-0.545455,0.0,0.0,0.0,0.0
3,SF,qb,ARI,4,9.272727,8.181818,74.636364,1.181818,-0.818182,6.0,14.272727,-0.545455,0.0,0.0,0.0,0.0
4,CIN,qb,ARI,5,0.272727,-4.818182,-24.363636,1.181818,1.181818,-3.0,-32.727273,-0.545455,0.0,0.0,0.0,0.0


In [13]:
df_strength = df_strength_raw.copy().drop(columns=['team', 'week'])
df_strength = df_strength.groupby(by=['opp', 'pos']).mean()
df_strength = df_strength.reset_index()
df_strength['fpts'] = (df_strength['pyd'] * 0.04) + (df_strength['ptd'] * 5) + (df_strength['int'] * -2) + (df_strength['rshyd'] * 0.1) + (df_strength['rshtd'] * 6) + (df_strength['rec'] * 0.5) + (df_strength['recyd'] * 0.1) + (df_strength['rectd'] * 6)
df_strength.head()

,opp,pos,att,cmp,pyd,ptd,int,rsh,rshyd,rshtd,targ,rec,recyd,rectd,fpts
0,ARI,qb,-2.927273,-0.392562,-4.283471,0.224793,-0.033058,-0.273554,-1.338843,0.189256,0.000000,0.000000,0.000000,0.000000,2.020397
1,ARI,rb,0.000000,0.000000,0.000000,0.000000,0.000000,4.310744,20.322314,0.319008,0.067769,0.514050,-2.572727,0.152893,4.863388
2,ARI,te,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.216364,-1.032727,-13.122727,-0.030000,-2.008636
3,ARI,wr,0.000000,0.000000,0.000000,0.000000,0.000000,-0.327273,-1.754545,-0.061983,-1.424793,0.458678,13.091736,0.108264,1.640744
4,ATL,qb,-2.751818,-2.322727,-29.365455,0.314545,-0.255455,1.508182,9.208182,0.210000,0.000000,0.000000,0.000000,0.000000,3.089836


In [14]:
load_dotenv()
df_strength.to_csv(f'{os.getenv('FILES_DIR')}/df_strength.csv')